### NOTE: Do not run this script for a concept scenario.

In [1]:
import openmatrix as omx
import pandas as pd
import numpy as np
import yaml
from utility import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
summary_dir = params['summary_dir']

time_periods = params['periods']
perf_measure_columns = params['final_columns']

best_path_skim_extension = params['best_path_skim_extension']

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [4]:
perf_measure_columns = perf_measure_columns
perf_measure_columns

['Concept_ID',
 'Metric_ID',
 'Metric_name',
 'Submetric',
 'Description',
 'Population',
 'Period',
 'Geography',
 'Zone_ID',
 'Origin_zone',
 'Dest_zone',
 'Purpose',
 'Mode',
 'Income',
 'Value',
 'Units',
 'Total_Increment']

## Final Code

In [5]:
tod_skims = pd.read_parquet(_join(preprocess_dir, 'tod_min_perceived_travel_time.parquet'))
# tod_skims.head()
tod_skims = tod_skims.reset_index()
tod_skims.head(2)

tp,orig,dest,am,ea,ev,md,pm
0,1,1,NaN,NaN,NaN,NaN,NaN
1,1,2,9359.0,7375.0,7343.0,7735.0,9336.0


In [6]:
tod_ivt = pd.read_parquet(_join(preprocess_dir, 'tod_min_ivt.parquet'))
# tod_ivt.head()
tod_ivt = tod_ivt.reset_index()
tod_ivt.head(2)

tp,orig,dest,am,ea,ev,md,pm
0,1,1,NaN,NaN,NaN,NaN,NaN
1,1,2,153.0,213.0,112.0,138.0,150.0


In [7]:
all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))

In [8]:
all_trips.head(2)

,hh_id,person_id,inbound,orig_purpose,dest_purpose,orig_taz,dest_taz,depart_hour,trip_mode,sampleRate,trip_type,trips,transbay_od,orig_rdm_zones,orig_super_dist,orig_county,dest_rdm_zones,dest_super_dist,dest_county,home_zone,income,Income,pp_share,link21_trip_purp,Period,Mode
0,2158574,5014593.0,0,Work,atwork,1,600,11,2,1.0,INM,1.0,0.0,San Francisco_43,4,1,San Francisco_43,4,1,1,154275,100k+,0.0,work,md,Auto_2Person
1,2158570,5014580.0,0,Work,atwork,645,161,14,3,1.0,INM,1.0,0.0,San Mateo_09,5,2,San Francisco_23,2,1,1,221062,100k+,0.0,work,md,Auto_3+Person


In [9]:
connected_ods = {}
final_trips = []
for p in time_periods:
    tp_df = all_trips[all_trips['Period'] == p.lower()]
    
    tod_ivt_df =  tod_ivt[['orig', 'dest', p.lower()]]
    tod_ivt_df = tod_ivt_df.dropna()
    tod_ivt_df.columns = ['orig_taz', 'dest_taz', 'ivt']
    print(f'Connected ODs in {p}: {len(tod_ivt_df):,}')
    connected_ods[p] = len(tod_ivt_df)
    
    tod_skims_df =  tod_skims[['orig', 'dest', p.lower()]]    
    tod_skims_df.columns = ['orig_taz', 'dest_taz', 'perc_tt']
    
    tod_skims_df = tod_skims_df.merge(tod_ivt_df, on=['orig_taz', 'dest_taz'], how='inner')
    
    tp_df = pd.merge(tp_df, tod_skims_df, on=['orig_taz', 'dest_taz'], how='inner')
    final_trips.append(tp_df)

Connected ODs in am: 10,877,905
Connected ODs in md: 10,789,736
Connected ODs in pm: 10,807,235
Connected ODs in ev: 10,778,704
Connected ODs in ea: 10,878,162


In [10]:
df_connected_ods = pd.DataFrame({'Period': connected_ods.keys(), 'Connected_ODs': connected_ods.values()})
df_connected_ods.to_csv(_join(summary_dir,
                              'A1.1'+ '_Connected_ODs_' + concept_id + '_region' + filename_extension + '.csv'),
                        index=None)

In [11]:
final_trips = pd.concat(final_trips)
final_trips['perc_tt'] = final_trips['perc_tt']/100

In [12]:
final_trips[1:5]

,hh_id,person_id,inbound,orig_purpose,dest_purpose,orig_taz,dest_taz,depart_hour,trip_mode,sampleRate,trip_type,trips,transbay_od,orig_rdm_zones,orig_super_dist,orig_county,dest_rdm_zones,dest_super_dist,dest_county,home_zone,income,Income,pp_share,link21_trip_purp,Period,Mode,perc_tt,ivt
1,986863,2417717.0,0,Work,atwork,705,637,9,2,1.0,INM,1.0,0.0,San Mateo_05,5,2,San Mateo_11,5,2,2515,35802,30k_to_60k,0.0,work,am,Auto_2Person,91.07,2405.0
2,2356245,5592509.0,1,eatout,Home,705,637,9,1,1.0,INM,1.0,0.0,San Mateo_05,5,2,San Mateo_11,5,2,637,68952,60k_to_100k,0.0,social,am,Auto_SOV,91.07,2405.0
3,2357297,5595752.0,1,social,Home,705,637,9,1,1.0,INM,1.0,0.0,San Mateo_05,5,2,San Mateo_11,5,2,637,66592,60k_to_100k,0.0,social,am,Auto_SOV,91.07,2405.0
4,2357561,5596501.0,1,othmaint,Home,705,637,6,2,1.0,INM,1.0,0.0,San Mateo_05,5,2,San Mateo_11,5,2,637,83321,60k_to_100k,0.0,othmaint,am,Auto_2Person,91.07,2405.0


In [14]:
final_trips_to_save = final_trips.copy()
final_trips_to_save.drop(columns=['hh_id', 'person_id', 'orig_purpose',
                                  'dest_purpose', 'depart_hour', 'sampleRate', 'income',
                                  'perc_tt', 'ivt'], inplace=True)
final_trips_to_save.to_parquet(_join(preprocess_dir, 'trip_roster_baseline_for_A1.parquet'))
del final_trips_to_save

In [15]:
summary_cols = params['description_a1.1']
summary_cols

{'perc_tt': ['perceived total travel time',
  'A1.1',
  'minutes',
  'perceived travel travel time']}

In [16]:
pp_df = []
reg_df = []
county_df = []
sd_df = []
tb_df = []
rdm_df = []
pp_df_rdm = []

for col in summary_cols:

    #for period in time_periods:
    #print(period)

    df_temp = final_trips# [final_trips['Period'] == period.lower()]

    df_temp[col + '_trips'] = df_temp[col] * df_temp['trips']

    #regional value
    reg_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column=col+'_trips')
    reg_df_temp1 = reg_df_temp1.rename(columns={'Value': col+'_trips'})
    
    reg_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='trips')
    reg_df_temp2 = reg_df_temp2.rename(columns={'Value': 'trips'})
    reg_df_temp = pd.merge(reg_df_temp1, reg_df_temp2, on=['Period', 'Income'], how='left')
    reg_df_temp[col] = reg_df_temp[col+'_trips']/reg_df_temp['trips']
    reg_df_temp = reg_df_temp[['Period', 'Income', col]]
    reg_df_temp = reg_df_temp.rename(columns={col:'Value'})
    reg_df_temp['Population'] = 'Whole Population'
    reg_df_temp['Origin_zone'] = ''
    reg_df_temp['Dest_zone'] = ''
    reg_df_temp['Zone_ID'] = ''
    reg_df_temp['Concept_ID'] = concept_id
    reg_df_temp['Geography'] = 'Regional'
    reg_df_temp['Purpose'] = ''
    reg_df_temp['Mode'] = ''
    reg_df_temp['Total_Increment'] = ''
    reg_df_temp['Metric_ID'] = summary_cols[col][1]
    reg_df_temp['Submetric'] = summary_cols[col][1]+'.1'
    reg_df_temp['Description'] = 'Average ' + summary_cols[col][0]+ ' in the region'
    reg_df_temp['Units'] = summary_cols[col][2]
    reg_df_temp['Metric_name'] = summary_cols[col][3]
    reg_df.append(reg_df_temp)
    print('Regional summary was performed.')
        
    # transbay region
    tb_df_t = df_temp[df_temp['transbay_od']==1]
    tb_df_temp = tb_df_t.copy()
    #tb_df_temp1 = tb_df_temp.groupby(['Period', 'Income'])[col+'_trips'].sum().reset_index()
    #tb_df_temp2 = tb_df_temp.groupby(['Period', 'Income'])['trips'].sum().reset_index()
    
    tb_df_temp1 = summarize_all_combinations(tb_df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column=col+'_trips')
    tb_df_temp1 = tb_df_temp1.rename(columns={'Value': col+'_trips'})
    
    tb_df_temp2 = summarize_all_combinations(tb_df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='trips')
    tb_df_temp2 = tb_df_temp2.rename(columns={'Value': 'trips'})
    
    tb_df_temp = pd.merge(tb_df_temp1, tb_df_temp2, on=['Period', 'Income'], how='left')
    tb_df_temp[col] = tb_df_temp[col+'_trips']/tb_df_temp['trips']
    tb_df_temp = tb_df_temp[['Period', 'Income', col]]
    tb_df_temp = tb_df_temp.rename(columns={col:'Value'})
 
    tb_df_temp['Concept_ID'] = concept_id
    tb_df_temp['Population'] = 'Whole Population'
    tb_df_temp['Origin_zone'] = ''
    tb_df_temp['Dest_zone'] = ''
    tb_df_temp['Zone_ID'] = ''
    tb_df_temp['Purpose'] = ''
    tb_df_temp['Mode'] = ''
    tb_df_temp['Geography'] = 'Transbay'
    tb_df_temp['Total_Increment'] = ''
    tb_df_temp['Metric_ID'] = summary_cols[col][1]
    tb_df_temp['Submetric'] = summary_cols[col][1]+'.2'
    tb_df_temp['Description'] = 'Average' + summary_cols[col][0] + ' in the transbay region'
    tb_df_temp['Units'] = summary_cols[col][2]
    tb_df_temp['Metric_name'] = summary_cols[col][3]

    tb_df.append(tb_df_temp)
    print('Transbay summary was performed.')
        
    # county 
    # county_df_temp1 = df_temp.groupby(['orig_county', 'dest_county', 'Period', 'Income'])[col+'_trips'].sum().reset_index()
    # county_df_temp2 = df_temp.groupby(['orig_county', 'dest_county', 'Period', 'Income'])['trips'].sum().reset_index()
    
    county_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['orig_county', 'dest_county', 'Period', 'Income'], 
                                                  summary_column=col+'_trips')
    county_df_temp1 = county_df_temp1.rename(columns={'Value': col+'_trips'})
    
    county_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['orig_county', 'dest_county', 'Period', 'Income'], 
                                                  summary_column='trips')
    county_df_temp2 = county_df_temp2.rename(columns={'Value': 'trips'})
    
    county_df_temp = pd.merge(county_df_temp1, county_df_temp2, on=['orig_county', 'dest_county', 'Period', 'Income'], how='left')
    county_df_temp[col] = county_df_temp[col+'_trips']/county_df_temp['trips']
    county_df_temp = county_df_temp[['orig_county', 'dest_county', 'Period', 'Income' , col]]
    county_df_temp = county_df_temp.rename(columns={col:'Value', 
                                                    'orig_county': 'Origin_zone',
                                                    'dest_county': 'Dest_zone'})
    county_df_temp['Population'] = 'Whole Population'
    county_df_temp['Zone_ID'] = ''
    county_df_temp['Concept_ID'] = concept_id
    county_df_temp['Geography'] = 'County'
    county_df_temp['Purpose'] = ''
    county_df_temp['Mode'] = ''
    county_df_temp['Total_Increment'] = ''
    county_df_temp['Description'] = 'Average ' + summary_cols[col][0] + ' in the origin and destination county'
    county_df_temp['Metric_ID'] = summary_cols[col][1]
    county_df_temp['Submetric'] = summary_cols[col][1]+'.3'
    county_df_temp['Units'] = summary_cols[col][2]
    county_df_temp['Metric_name'] = summary_cols[col][3]
    county_df.append(county_df_temp)
    print('County summary was performed.')


    #RDM Zones
    #rdm_df_temp1 = df_temp.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'])[col+'_trips'].sum().reset_index()
    #rdm_df_temp2 = df_temp.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'])['trips'].sum().reset_index()
    
    rdm_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], 
                                                  summary_column=col+'_trips')
    rdm_df_temp1 = rdm_df_temp1.rename(columns={'Value': col+'_trips'})
    
    rdm_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], 
                                                  summary_column='trips')
    rdm_df_temp2 = rdm_df_temp2.rename(columns={'Value': 'trips'})
    
    rdm_df_temp = pd.merge(rdm_df_temp1, rdm_df_temp2, on=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], how='left')
    rdm_df_temp[col] = rdm_df_temp[col+'_trips']/rdm_df_temp['trips']
    rdm_df_temp = rdm_df_temp[['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income', col]]
    rdm_df_temp = rdm_df_temp.rename(columns={'orig_rdm_zones': 'Origin_zone', 
                                              'dest_rdm_zones': 'Dest_zone', 
                                              col: 'Value'})
    rdm_df_temp['Population'] = 'Whole Population'
    rdm_df_temp['Zone_ID'] = ''
    rdm_df_temp['Geography'] = 'RDM'
    rdm_df_temp['Concept_ID'] = concept_id
    rdm_df_temp['Purpose'] = ''
    rdm_df_temp['Mode'] = ''
    rdm_df_temp['Total_Increment'] = ''
    rdm_df_temp['Description'] = 'Average ' + summary_cols[col][0] + ' in the origin and destination RDM zones'  
    rdm_df_temp['Metric_ID'] = summary_cols[col][1]
    rdm_df_temp['Submetric'] = summary_cols[col][1]+'.4'
    rdm_df_temp['Units'] = summary_cols[col][2]
    rdm_df_temp['Metric_name'] = summary_cols[col][3]
    rdm_df.append(rdm_df_temp)
    print('RDM zones summary was performed.')
        
    # super district        
    #sd_df_temp1 = df_temp.groupby(['orig_super_dist', 'dest_super_dist', 'Period', 'Income'])[col+'_trips'].sum().reset_index()
    #sd_df_temp2 = df_temp.groupby(['orig_super_dist', 'dest_super_dist', 'Period', 'Income'])['trips'].sum().reset_index()
    
    sd_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['orig_super_dist', 'dest_super_dist', 'Period', 'Income'], 
                                                  summary_column=col+'_trips')
    sd_df_temp1 = sd_df_temp1.rename(columns={'Value': col+'_trips'})
    
    sd_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['orig_super_dist', 'dest_super_dist', 'Period', 'Income'], 
                                                  summary_column='trips')
    sd_df_temp2 = sd_df_temp2.rename(columns={'Value': 'trips'})
      
    sd_df_temp = pd.merge(sd_df_temp1, sd_df_temp2, on=['orig_super_dist', 'dest_super_dist', 'Period', 'Income'], how='left')
    sd_df_temp[col] = sd_df_temp[col+'_trips']/sd_df_temp['trips']
    sd_df_temp = sd_df_temp[['orig_super_dist', 'dest_super_dist', 'Period', 'Income', col]]
    sd_df_temp = sd_df_temp.rename(columns={'orig_super_dist': 'Origin_zone', 
                                            'dest_super_dist': 'Dest_zone',
                                             col: 'Value'})
    sd_df_temp['Population'] = 'Whole Population'
    sd_df_temp['Zone_ID'] = ''
    sd_df_temp['Concept_ID'] = concept_id
    sd_df_temp['Purpose'] = ''
    sd_df_temp['Mode'] = ''
    sd_df_temp['Total_Increment'] = ''
    sd_df_temp['Geography'] = 'Super district'
    sd_df_temp['Description'] = 'Average '+ summary_cols[col][0] + ' in the origin and destination Super district'  
    sd_df_temp['Metric_ID'] = summary_cols[col][1]
    sd_df_temp['Submetric'] = summary_cols[col][1]+'.5'
    sd_df_temp['Units'] = summary_cols[col][2]
    sd_df_temp['Metric_name'] = summary_cols[col][3]
    sd_df.append(sd_df_temp)
    print('Super district summary was performed.')
        
    # prioirty population
    df_temp['pp_share'] = df_temp['pp_share']/100
    df_temp['pp_trips'] = df_temp['pp_share'] * df_temp['trips']
    df_temp['pp_'+col] = df_temp['pp_trips'] * df_temp[col] #/ 100

    #pp_df_temp1 = df_temp.groupby(['Period', 'Income'])['pp_'+col].sum().reset_index()
    #pp_df_temp2 = df_temp.groupby(['Period', 'Income'])['pp_trips'].sum().reset_index()
    
    pp_rdm_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], 
                                                  summary_column='pp_'+col)
    pp_rdm_df_temp1 = pp_rdm_df_temp1.rename(columns={'Value': 'pp_'+col})
    
    pp_rdm_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], 
                                                  summary_column='pp_trips')
    pp_rdm_df_temp2 = pp_rdm_df_temp2.rename(columns={'Value': 'pp_trips'})
    
    pp_rdm_df_temp = pd.merge(pp_rdm_df_temp1, pp_rdm_df_temp2, on=['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income'], how='left')
    pp_rdm_df_temp[col] = pp_rdm_df_temp['pp_'+col]/pp_rdm_df_temp['pp_trips']
    pp_rdm_df_temp = pp_rdm_df_temp[['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Income', col]]
    
    pp_rdm_df_temp = pp_rdm_df_temp.rename(columns={col:'Value', 
                                                   'orig_rdm_zones': 'Origin_zone', 
                                                    'dest_rdm_zones': 'Dest_zone'})
    
    pp_rdm_df_temp['Population'] = 'Prioirty Population'
    #pp_rdm_df_temp['Origin_zone'] = ''
    #pp_rdm_df_temp['Dest_zone'] = ''
    pp_rdm_df_temp['Zone_ID'] = ''
    pp_rdm_df_temp['Purpose'] = ''
    pp_rdm_df_temp['Mode'] = ''
    pp_rdm_df_temp['Concept_ID'] = concept_id
    pp_rdm_df_temp['Geography'] = 'RDM'
    pp_rdm_df_temp['Total_Increment'] = ''
    pp_rdm_df_temp['Metric_ID'] = summary_cols[col][1]
    pp_rdm_df_temp['Submetric'] = summary_cols[col][1]+'.7'
    pp_rdm_df_temp['Description'] = 'Average ' + summary_cols[col][0] + ' in the region'
    pp_rdm_df_temp['Units'] = summary_cols[col][2]
    pp_rdm_df_temp['Metric_name'] = summary_cols[col][3]
    pp_df_rdm.append(pp_rdm_df_temp)
    
    
    pp_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='pp_'+col)
    pp_df_temp1 = pp_df_temp1.rename(columns={'Value': 'pp_'+col})
    
    pp_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='pp_trips')
    pp_df_temp2 = pp_df_temp2.rename(columns={'Value': 'pp_trips'})
    
    pp_df_temp = pd.merge(pp_df_temp1, pp_df_temp2, on=['Period', 'Income'], how='left')
    pp_df_temp[col] = pp_df_temp['pp_'+col]/pp_df_temp['pp_trips']
    pp_df_temp = pp_df_temp[['Period', 'Income', col]]
    pp_df_temp = pp_df_temp.rename(columns={col:'Value'})
    
    pp_df_temp['Population'] = 'Prioirty Population'
    pp_df_temp['Origin_zone'] = ''
    pp_df_temp['Dest_zone'] = ''
    pp_df_temp['Zone_ID'] = ''
    pp_df_temp['Purpose'] = ''
    pp_df_temp['Mode'] = ''
    pp_df_temp['Concept_ID'] = concept_id
    pp_df_temp['Geography'] = 'Regional'
    pp_df_temp['Total_Increment'] = ''
    pp_df_temp['Metric_ID'] = summary_cols[col][1]
    pp_df_temp['Submetric'] = summary_cols[col][1]+'.6'
    pp_df_temp['Description'] = 'Average ' + summary_cols[col][0] + ' in the region'
    pp_df_temp['Units'] = summary_cols[col][2]
    pp_df_temp['Metric_name'] = summary_cols[col][3]
    pp_df.append(pp_df_temp)
    print('Prioirty population summary was performed.')

        
pp_df = pd.concat(pp_df)
reg_df = pd.concat(reg_df)
county_df = pd.concat(county_df)
sd_df = pd.concat(sd_df)
rdm_df = pd.concat(rdm_df)
tb_df = pd.concat(tb_df)
pp_rdm_df = pd.concat(pp_df_rdm)

Regional summary was performed.
Transbay summary was performed.
County summary was performed.
RDM zones summary was performed.
Super district summary was performed.
Prioirty population summary was performed.


In [17]:
#reg_df

In [18]:
all_dfs = [reg_df, tb_df, county_df, rdm_df, sd_df, pp_df, pp_rdm_df]

for dfs in all_dfs:
    metric_name = '_perceived_travel_time_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0], dfs['Value'].sum())
    
combined_df = pd.concat([reg_df, tb_df, county_df, rdm_df, sd_df, pp_df]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'A1.1'+ '_perceived_travel_time_' + concept_id + '_region' + filename_extension + '.csv'), index=None)

30 A1.1.1 perceived travel travel time 2972.4563852385413
30 A1.1.2 perceived travel travel time 2802.1808941027593
2900 A1.1.3 perceived travel travel time 542469.4980239914
1088844 A1.1.4 perceived travel travel time 126949087.42733495
31502 A1.1.5 perceived travel travel time 5469581.109416021
30 A1.1.6 perceived travel travel time 3343.4428649925726
1088844 A1.1.7 perceived travel travel time 73781395.96010818


In [19]:
#county_df